# 2)  Creacion de dataset prediccion

### Instalacion de pysatproc (correr solo una vez)

In [7]:
#!pip install -U pysatproc

In [12]:
!ls "../images-S2/zona_4/2021-01-01_2021-03-01/"

0000000000-0000000000.tif  0000032768-0000065536.tif  0000081920-0000049152.tif
0000000000-0000016384.tif  0000049152-0000000000.tif  0000081920-0000065536.tif
0000000000-0000032768.tif  0000049152-0000016384.tif  0000098304-0000000000.tif
0000000000-0000049152.tif  0000049152-0000032768.tif  0000098304-0000016384.tif
0000000000-0000065536.tif  0000049152-0000049152.tif  0000098304-0000032768.tif
0000016384-0000000000.tif  0000049152-0000065536.tif  0000098304-0000049152.tif
0000016384-0000016384.tif  0000065536-0000000000.tif  0000098304-0000065536.tif
0000016384-0000032768.tif  0000065536-0000016384.tif  0000114688-0000000000.tif
0000016384-0000049152.tif  0000065536-0000032768.tif  0000114688-0000016384.tif
0000016384-0000065536.tif  0000065536-0000049152.tif  0000114688-0000032768.tif
0000032768-0000000000.tif  0000065536-0000065536.tif  0000114688-0000049152.tif
0000032768-0000016384.tif  0000081920-0000000000.tif  0000114688-0000065536.tif
0000032768-0000032768.tif  0000081920-00

In [26]:
#indico path a imagenes de prediccion y donde guardo el dataset. Tambien, size y step size.
path_to_files = "../images-S2/zona_4/2021-01-01_2021-03-01/*.tif"            # Path to the satellite images folder
output_folder = "../dataset/data_predict/zona_4/v2/160_80/"        # Destination path <ndim>/<size>_<step-size>/
#output_folder = "../dataset/test_borrar"  
vector_aoi_file_path = "../data/shp/predict_aoi_zona4_buff5km_4326.geojson"
                       
size = 160            # size (pixels) of each dataset image generated
step_size = 160        # step  size of the sliding window 

In [27]:
#Creo dataset para prediccion (solo imagenes)
!satproc_extract_chips $path_to_files \
    -o $output_folder \
    --aoi $vector_aoi_file_path \
    --size $size \
    --step-size $step_size \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100


0000000000-0000016384.tif windows:  43%|#7  | 430/1007 [00:03<00:05, 105.83it/s]


0000016384-0000016384.tif windows:  20%|7   | 292/1490 [00:02<00:11, 100.12it/s]


0000032768-0000000000.tif windows:  95%|##8| 1500/1583 [00:13<00:00, 107.62it/s]


Rasters:  90%|###############################5   | 9/10 [01:44<00:07,  7.51s/it]


Rasters: 100%|##################################| 10/10 [01:45<00:00, 10.56s/it]


# 3) Prediccion

In [28]:
#importo librerias
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_results
import os

In [13]:
#!ls ../data/weights/model/

In [29]:
#Tengo que llamar al model_path para tener el modelo y a las img_pred para tener las imagenes
predict_config = PredictConfig(
                                    images_path=os.path.join('../dataset/data_predict','zona_4','v2','160_80'),
                                    results_path=os.path.join('../dataset/data_results','zona_4','v2','160_80'),
                                    batch_size=16,
                                    model_path='../data/weights/model/zona_3UNet_techo_4D_spe100_img160_size160_sz80.h5',  #  ruta al modelo (.h5)
                                    height=160,
                                    width=160,
                                    n_channels=4,
                                    n_classes=1,
                                    class_weights=[1])

In [30]:
predict(predict_config)

Total images to predict (../dataset/data_predict/zona_4/v2/160_80/images/*.tif): 10794
After skipping existing results: 10794


  0%|          | 0/675 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


100%|██████████| 675/675 [20:42<00:00,  1.84s/it]

Done!


In [10]:
#plot_data_results(predict_config=predict_config, num_samples=3, fig_size=(20,10), img_ch =2, n_bands = 1)

TypeError: plot_data_results() got an unexpected keyword argument 'n_bands'

# 4) Post

In [31]:
from satproc.postprocess.polygonize import polygonize 
from satproc.filter import filter_by_max_prob


In [32]:
!mkdir -p ../dataset/data_results/zona_4/v2/160_80_filtered_up03/

In [33]:
#filtra las imagenes con prob mayor al parametro
filter_by_max_prob(input_dir=os.path.join('../dataset/data_results','zona_4','v2','160_80'),
                   output_dir='../dataset/data_results/zona_4/v2/160_80_filtered_up03/',
                   threshold=0.3) #Probar ≠ prob umbrales, por ahi, empezar con 0.1

100%|##########| 10794/10794 [00:25<00:00, 429.58it/s]


In [34]:
input_path =  '../dataset/data_results/zona_4/v2/160_80_filtered_up03/'
output_path = '../dataset/data_results/zona_4/v2_160_80_zona4_thr03.gpkg' 


polygonize(threshold=0.3,
          # value=100,
           input_dir=input_path,
           output=output_path)
#______________________________________

Dissolve: 100%|##########| 55/55 [00:43<00:00,  1.27it/s]


In [35]:
src_file = '../dataset/data_results/zona_4/v2_160_80_zona4_thr03.gpkg'
dst_file = '../dataset/data_results/zona_4/v2_160_80_zona4_thr03_utm.gpkg'

!ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:32720 -f 'ESRI Shapefile' $dst_file $src_file #lo puedo abrir en QGIS como archivo vectorial

In [36]:
min_area = 500 #mas grande que una manzana (son 7000 m2)
input_path = "../dataset/data_results/zona_4/v2_160_80_zona4_thr03_utm.gpkg" #decia _utm -->esta bien tiene que estar en metros para hacer operaciones de area
output_path = "../dataset/data_results/zona_4/v2_160_80_zona4_thr03_utm_up500.gpkg"

In [15]:
!ogrinfo -al $input_path | head -n25

INFO: Open of `../dataset/data_results/zona_2/v2_160_80_zona2_thr03_utm.gpkg'
      using driver `ESRI Shapefile' successful.

Layer name: v2_160_80_zona2_thr03
Metadata:
  DBF_DATE_LAST_UPDATE=2022-03-09
Geometry: Polygon
Feature Count: 12677
Extent: (-323037.315383, 6408638.178516) - (-156476.348487, 7085903.797681)
Layer SRS WKT:
PROJCRS["WGS 84 / UTM zone 20S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 20S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["Degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-63,


In [37]:
!ogr2ogr \
    -t_srs EPSG:32720 \
    -f "GPKG" \
    -sql "SELECT * FROM v2_160_80_zona4_thr03 m WHERE ST_Area(geometry) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $output_path \
    $input_path

In [41]:
#!ogrinfo ../data/post/anot_nuevas_mod/zona_3/zona_3_160x160_thr01_utm_up500.gpkg
#!ogrinfo -so ./data/post/anot_nuevas_mod/zona_3/zona_3_160x160_thr01_utm_up500.gpkg results

# -----------------------------------

## Extraigo tabla de areas predichas de los poligonos

In [ ]:
import geopandas
import statistics as st

path_utm = "post/anot_nuevas/zona_3/modelo_320_20/zona_3_320x320_thr01.gpkg"


gdf = geopandas.read_file(path_utm)
print(gdf)
print(gdf["area"])
print(st.mean(gdf["area"]))
#gdf["area"] = gdf.area
gdf["area"].plot.hist(bins=20, range=(0,10000))


### Funcion para filtrar todos los archivos gpk de un directorio

In [3]:
import os

def pasar_utm(archivo):
    src_file = archivo
    dst_file = archivo.replace(".gpkg", "_utm.gpkg")
    print('archivo fuente: '+src_file)
    print('archivo utm: '+dst_file)
    !ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:32720 -f 'GPKG' $dst_file $src_file #lo puedo abrir en QGIS como archivo vectorial


def filtrar_poligonos(archivo, min_area):
    ##pasasje a utm
    #src_file = archivo
    #dst_file = archivo.replace(".gpkg", "_utm.gpkg")
    ##print('archivo fuente: '+src_file)
    ##print('archivo utm: '+dst_file)
    #!ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:32720 -f 'GPKG' $dst_file $src_file #lo puedo abrir en QGIS como archivo vectorial
    pasar_utm(archivo)
    
    #filtrado
    area = min_area #500 #mas grande que una manzana (son 7000 m2)
    input_path = dst_file#"../archivos_de_la_VM/predicciones_finales/join_03_utm.gpkg" #decia _utm -->esta bien tiene que estar en metros para hacer operaciones de area
    output_path = dst_file.replace(".gpkg", "_"+str(min_area)+".gpkg")  
    tabla = archivo.split("/",-1)[-1].strip('.gpkg')
    sql = "SELECT * FROM "+tabla+" m WHERE ST_Area(geom) > "+str(min_area)
    
    #print('archivo filtrado: '+output_path)
    #print('sql: '+sql)
    #print('output_path: '+output_path)
    #print('input_path: '+input_path)
    
    !ogr2ogr -t_srs EPSG:32720 -f "GPKG" -sql "$sql" -dialect SQLITE -nln results $output_path $input_path
    
    !rm $input_path #para borrar el archivo_utm que no se usa
        
def filtrar_todos_los_poligonos(directorio, min_area):
    import glob
    directorio_gpk = directorio+"/*.gpkg"
    listing = glob.glob(directorio_gpk)
    for filename in listing:
        print('procesando: '+filename) 
        filtrar_poligonos(filename, min_area)
    


### Funcion para generar gpkq para emular un heatmap

In [63]:
def heatmap(carpeta):
    import os
    from satproc.postprocess.polygonize import polygonize 
    from satproc.filter import filter_by_max_prob

    for i in range(1,11,1):
        thr=i/10
        print('heatmap ', str(thr))
        #filtra las imagenes con prob mayor al parametro
        
        nueva_carpeta = carpeta.replace(carpeta.split("/",-1)[-1],'')+'heatmap_'+str(thr)
        !mkdir -p $nueva_carpeta

        filter_by_max_prob(input_dir=carpeta,
                           output_dir=nueva_carpeta,
                           threshold=thr) 

        input_path =  nueva_carpeta
        output_path = '../dataset/data_results/zona_1/'+'heatmap_'+str(thr)+'.gpkg' 
        polygonize(threshold=thr,
                   value=100,
                   input_dir=input_path,
                   output=output_path)


# ----------------------------------------------

In [12]:
pasar_utm('../dataset/data_results/zona_3/finales/join_05.gpkg')

archivo fuente: ../dataset/data_results/zona_3/finales/join_05.gpkg
archivo utm: ../dataset/data_results/zona_3/finales/join_05_utm.gpkg


In [131]:
filtrar_todos_los_poligonos('../dataset/data_results/zona_3/finales',1000)

procesando: ../dataset/data_results/zona_3/finales/v2_160_60_zona3_thr05_final.gpkg
procesando: ../dataset/data_results/zona_3/finales/v2_160_60_zona3_thr03_final.gpkg
procesando: ../dataset/data_results/zona_3/finales/v2_100_50_zona3_thr03_final.gpkg
procesando: ../dataset/data_results/zona_3/finales/join_03.gpkg
procesando: ../dataset/data_results/zona_3/finales/v2_100_50_zona3_thr05_final.gpkg
procesando: ../dataset/data_results/zona_3/finales/join_05.gpkg


In [17]:
!ls '../data/shp/gt/R3'

3_Macrozona_Centro
Asentamiento_R3.geojson
Asentamiento_R3_nuevo.geojson
Asentamiento_R3_nuevo_100_puntos.geojson
Asentamiento_R3_nuevo_100_puntos_buffer.geojson
Asentamiento_R3_v2.geojson
Asentamiento_R3_valentin_v1.geojson
Asentamientos_R3_nuevo_100_puntos_buffer.gpkg
Asentamientos_R3_nuevo_100_puntos_buffer.gpkg-shm
Asentamientos_R3_nuevo_100_puntos_buffer.gpkg-wal
Heatmap
NO_asentamiento_R3_nuevo_100_puntos_buffer.geojson
int_100_50_th03_100a.geojson
int_100_50_th05_100a.geojson
int_100_50_th05_100noa.geojson
int_100_50_the03_100_noa.geojson
int_160_60_thr03_100a.geojson
int_160_60_thr03_100noa.geojson
int_160_60_thr05_100_noa.geojson
int_160_60_thr05_100a.geojson


In [64]:
heatmap('../dataset/data_results/zona_1/v2/160_80')

heatmap  0.1


Dissolve: 100%|##########| 3/3 [00:18<00:00,  6.17s/it]


heatmap  0.2


Dissolve: 100%|##########| 2/2 [00:10<00:00,  5.08s/it]


heatmap  0.3


...

Dissolve: 100%|##########| 1/1 [00:00<00:00,  1.64it/s]


heatmap  0.9


Dissolve: 100%|##########| 1/1 [00:00<00:00,  4.97it/s]


heatmap  1.0


Merge groups: 100%|##########| 1/1 [00:00<00:00, 15.50it/s]
No shapes. Will not write output file
